In [ ]:
%pip install openai requests --upgrade

In [6]:
import requests
import json
import re
import os
import requests
import getpass

In [7]:
OPENAI_API_KEY = getpass.getpass("Please enter your OpenAI Key:")

In [4]:
IS_RUNNING_CODE_LOCALLY = True

In [5]:
# Your Google Drive shared link
LINK = (
    "https://drive.google.com/file/d/1O6hubG33SyrG24m6TPZOLAmS0J2RxLCW/view?usp=sharing"
)

# Extract the file ID from the URL using regex
file_id_match = re.search(r"d/([a-zA-Z0-9_-]+)/", LINK)
if file_id_match:
    FILE_ID = file_id_match.group(1)
else:
    raise ValueError("Could not extract file ID from the provided URL")

# Generate the direct download link
URL = f"https://drive.google.com/uc?export=download&id={FILE_ID}"

# Download the file content
response = requests.get(URL)
response.raise_for_status()

# Save the content to a local .jsonl file in Colab
file_path = "/content/training_data.jsonl"
if IS_RUNNING_CODE_LOCALLY:
    file_path = "training_data.jsonl"

with open(file_path, "w") as f:
    f.write(response.text)

print(f"File saved to {file_path}")

File saved to training_data.jsonl


In [6]:
# List to hold all the JSON objects from the file
data_list = []

with open(file_path, 'r') as file:
    for line in file:
        data = json.loads(line.strip())
        data_list.append(data)

# Now data_list contains all the JSON objects from the file
len(data_list)

304

In [12]:
headers = {
    'Authorization': f'Bearer {OPENAI_API_KEY}',
}

files = {
    'file': (file_path.split('/')[-1], open(file_path, 'rb')),
}

data = {
    'purpose': 'fine-tune',
}

response = requests.post(
    'https://api.openai.com/v1/files',
    headers=headers,
    data=data,
    files=files
)

print(response.json())

{'object': 'file', 'id': 'file-U6aGVwKtwDA91Sqvw94umv', 'purpose': 'fine-tune', 'filename': 'training_data.jsonl', 'bytes': 464306, 'created_at': 1744220809, 'expires_at': None, 'status': 'processed', 'status_details': None}


In [34]:
# Check the file is loaded
response = requests.get(
    'https://api.openai.com/v1/files',
    headers=headers,
)
response.json()['data'][-1]

{'object': 'file',
 'id': 'file-U6aGVwKtwDA91Sqvw94umv',
 'purpose': 'fine-tune',
 'filename': 'training_data.jsonl',
 'bytes': 464306,
 'created_at': 1744220809,
 'expires_at': None,
 'status': 'processed',
 'status_details': None}

In [ ]:
import json

TRAINING_FILE_ID = response.json()['data'][-1]['id']

url = 'https://api.openai.com/v1/fine_tuning/jobs'

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {OPENAI_API_KEY}',
}

data = {
    "training_file": TRAINING_FILE_ID,
    "model": "gpt-3.5-turbo-1106",
}

response = requests.post(url, headers=headers, data=json.dumps(data))

print(response.json())

In [17]:
job_id = "ftjob-IJKWGuXaGbmk9iaDBwOo0dCR"
response = requests.get(
    f"https://api.openai.com/v1/fine_tuning/jobs/{job_id}",
    headers={
        "Authorization": f"Bearer {OPENAI_API_KEY}"
    }
)

In [20]:
print(response.json())  # Check status

{'object': 'fine_tuning.job', 'id': 'ftjob-IJKWGuXaGbmk9iaDBwOo0dCR', 'model': 'gpt-3.5-turbo-1106', 'created_at': 1744221114, 'finished_at': 1744222591, 'fine_tuned_model': 'ft:gpt-3.5-turbo-1106:personal::BKUGPfmw', 'organization_id': 'org-C5xNTgkE7xTz1VYEy6oKSbfM', 'result_files': ['file-AhTmbjisDigPwqrXoAH6Lc'], 'status': 'succeeded', 'validation_file': None, 'training_file': 'file-U6aGVwKtwDA91Sqvw94umv', 'hyperparameters': {'n_epochs': 3, 'batch_size': 1, 'learning_rate_multiplier': 2}, 'trained_tokens': 277545, 'error': {}, 'user_provided_suffix': None, 'seed': 1187546622, 'estimated_finish': None, 'integrations': [], 'metadata': None, 'method': {'type': 'supervised', 'supervised': {'hyperparameters': {'n_epochs': 3, 'batch_size': 1, 'learning_rate_multiplier': 2.0}}}}


In [24]:
response.json()['trained_tokens'] * 0.008 / 1000 # 0.008 cents per 1k tokens

2.2203600000000003

In [25]:
print(response.json()['fine_tuned_model']) # get the model name

ft:gpt-3.5-turbo-1106:personal::BKUGPfmw


In [27]:
MODEL_NAME = response.json()['fine_tuned_model']

url = 'https://api.openai.com/v1/chat/completions'

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {OPENAI_API_KEY}',
}

data = {
    "model": MODEL_NAME,
    "messages": [
        {
            "role": "system",
            "content": "You are a helpful assistant."
        },
        {
            "role": "user",
            "content": "Write the section 'How Agencies Make Money' for the blog post 'Agency Economics'"
        }
    ]
}

response = requests.post(url, headers=headers, data=json.dumps(data))

print(response.json())  # Print the response or handle it as needed

{'id': 'chatcmpl-BKUTI8PKpB5chMIOyTMdF4HXY6Hcn', 'object': 'chat.completion', 'created': 1744223392, 'model': 'ft:gpt-3.5-turbo-1106:personal::BKUGPfmw', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "It sounds mercenary to discuss it so plainly, but getting to understanding how agencies make their money without understanding how they work economically is your road to ruin. Agencies aren't non-profits — they are businesses and although none of them handle advertising themselves, they market for clients. Knowing the terminology can be really helpful in conversations, and can lead to you making a better deal economically. See more information about this.The Sandwich ProblemFor instance, many employees at agencies might be surprised to realize the impact they have on gross margins — many people see their names on the website and think they are directly 'billable', yet despite what they might complain to their significant other, they would never be billed out at 100%

In [28]:
print(response.json()['choices'][0]['message']['content'])

It sounds mercenary to discuss it so plainly, but getting to understanding how agencies make their money without understanding how they work economically is your road to ruin. Agencies aren't non-profits — they are businesses and although none of them handle advertising themselves, they market for clients. Knowing the terminology can be really helpful in conversations, and can lead to you making a better deal economically. See more information about this.The Sandwich ProblemFor instance, many employees at agencies might be surprised to realize the impact they have on gross margins — many people see their names on the website and think they are directly 'billable', yet despite what they might complain to their significant other, they would never be billed out at 100% of their cost. In fact, to get a margin of 20-40%, a billable employee must not have their direct cost be more than 50-80% of their market rate — leaving precious little margin for doing good work, taxes, or investment. Thi

In [29]:
data = {
    "model": MODEL_NAME,
    "messages": [
        {
            "role": "system",
            "content": "You are a helpful assistant."
        },
        {
            "role": "user",
            "content": "Write the section 'Criticisms of Memetics' for the blog post 'Why isn't Memetics a science?'"
        }
    ]
}

response = requests.post(url, headers=headers, data=json.dumps(data))

print(response.json()['choices'][0]['message']['content'])


By Sokanon – Own work (Original text: I created this work entirely by myself.), CC BY-SA 3.0, https://commons.wikimedia.org/w/index.php?curid=20706071Memes have been strongly criticised by biologists like Dawkins who believes they don't have the same properties as genes and therefore aren't a useful concept, other than describing the spread of internet jokes (or the effect on information of any infectious disease might be a more timely example).Dawkins himself says "I think it‘s possible to have a science of cultural evolution", then goes on to point out how hard it is to quantify memes and test their spread.  In his book The God Delusion, he says:  “It is socially useful to have an over-arching term, as a unifying discipline, for ‘the study of things that spread.’ But ‘viral marketing’ and ‘memetics’ are doing scientific and public understanding a disservice in lumping together such a mix of ill-matched study topics." ~ Richard Dawkins
